**Final**

In [7]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from collections import OrderedDict
import ipywidgets as widgets
from IPython.display import display, HTML


# Sample data with categories for each ingredient
data = {
    'Recipe': [
        'Rotti', 'Pasta', 'Pittu', 'Thosa', 'Idli', 'Vadai', 'Ulundhu Vadai', 'Uppuma', 'Hopper (Aappa)',
        'Noodles', 'Bun', 'Pizza', 'Patties', 'Pastry', 'Cake', 'Pancake', 'Briyani', 'Fried Rice',
        'Milk Rice (Kiri Badh)', 'Pongal', 'Doughnut', 'Kanji', 'Sandwich', 'String Hopper', 'Rice', 'Pulses'
    ],
    'Ingredients': [
        'Wheat Flour, Coconut, Butter',
        'Pasta, Carrot, Chicken, Egg, Soy Sauce, Tomato Sauce',
        'Rice Flour, Coconut',
        'Wheat Flour, Ulundhu, Butter, Baking Soda',
        'Rice, Ulundhu, Baking Soda',
        'Bengal Gram, Chilli, Onion',
        'Ulundhu, Rice Flour, Onion, Chilli',
        'Rava, Carrot, Onion',
        'Rice Flour, Coconut, Egg, Baking Soda',
        'Spaghetti, Egg, Soy Sauce, Carrot, Chicken, Tomato Sauce',
        'Wheat Flour, Egg, Butter, Yeast',
        'Wheat Flour, Egg, Butter, Cheese, Yeast, Tomato Sauce',
        'Wheat Flour, Egg, Butter, Potato, Onion',
        'Wheat Flour, Butter, Baking Powder',
        'Wheat Flour, Butter, Egg, Baking Powder, Sugar',
        'Wheat Flour, Milk, Egg, Baking Powder',
        'Rice, Chicken, Onion, Potato',
        'Rice, Carrot, Soy Sauce, Onion, Egg, Chicken',
        'Rice, Coconut, Sugar',
        'Rice, Coconut, Jaggery, Green Gram',
        'Wheat Flour, Milk, Butter, Yeast, Sugar',
        'Rice, Coconut, Chicken, Carrot',
        'Bread, Cheese, Chicken, Tomato Sauce',
        'Rice Flour',
        'Rice',
        'Chickpeas, Green Gram, Kidney Beans'
    ]
}

df = pd.DataFrame(data)

# Preprocess the data
df['Ingredients'] = df['Ingredients'].str.replace(',', ' ')

# Create a CountVectorizer to convert text into a matrix of token counts
vectorizer = CountVectorizer()
ingredient_matrix = vectorizer.fit_transform(df['Ingredients'])

# Compute cosine similarity
cosine_sim = cosine_similarity(ingredient_matrix, ingredient_matrix)

# Function to recommend recipes based on selected ingredients
def get_recipe_recommendations(selected_ingredients, cosine_sim, df):
    # Convert both selected ingredients and ingredients in DataFrame to lowercase
    selected_ingredients_lower = [ing.lower() if ing.lower() != 'not needed' else 'not needed' for ing in selected_ingredients]
    print("\nselected ingredients are :", selected_ingredients_lower)
    df['Ingredients_lower'] = df['Ingredients'].str.lower()

    # Extracting unique first ingredients in order
    main_ingredient = list(OrderedDict.fromkeys(ingredient.split(', ')[0] for ingredient in data['Ingredients']))
    print("\nmain ingredients are :", main_ingredient)

    if not any(ingredient.lower() in selected_ingredients_lower for ingredient in main_ingredient):
        print(f"\nThe main ingredient to make these recipes is not in the selected ingredients.")
        return [], []

    # Combine selected ingredients into a query
    query = ' '.join(selected_ingredients_lower)

    # Transform the query using the CountVectorizer
    query_vectorized = vectorizer.transform([query])

    # Compute cosine similarity between the query and recipes
    cosine_scores = cosine_similarity(query_vectorized, ingredient_matrix)

    # Get indices of recipes sorted by similarity
    recipe_indices = cosine_scores.argsort()[0][::-1]

    # Initialize a dictionary to store recipe scores based on matching ingredients
    recipe_scores = {}
    for index in recipe_indices:
        recipe = df['Recipe'].iloc[index]
        ingredients = df['Ingredients_lower'].iloc[index].split()
        matched_count = sum(1 for ing in selected_ingredients_lower if ing in ingredients)
        recipe_scores[recipe] = matched_count

    # Sort recipes based on the number of matching ingredients
    sorted_recipes = sorted(recipe_scores.items(), key=lambda x: x[1], reverse=True)

    # Return the top 5 recipes and the single best recipe
    top_5_recipes = [recipe[0] for recipe in sorted_recipes[:5]]
    single_best_recipe = [sorted_recipes[0][0]] if sorted_recipes else []

    return top_5_recipes, single_best_recipe





# Function to get user input and display recommendations

def get_user_input_and_display_recommendations():
    print("Available recipes:")
    print(df['Recipe'].values)

    # Sample ingredient categories
    categories = {
        'Proteins': ['Chicken', 'Egg', 'Not Needed'],
        'Vegetables': ['Carrot', 'Onion', 'Potato', 'Chilli', 'Coconut', 'Not Needed'],
        'Main': ['Wheat Flour', 'Pasta', 'Rice Flour', 'Rice', 'Bengal Gram', 'Ulundhu', 'Rava', 'Spaghetti',
                 'Bread', 'Chickpeas', 'Not Needed'],
        'Lentils': ['Green Gram', 'Kidney Beans', 'Not Needed'],
        'Dairy': ['Milk', 'Butter', 'Cheese', 'Not Needed'],
        'Leavening Agents': ['Baking Powder', 'Baking Soda', 'Yeast', 'Not Needed'],
        'Sauces': ['Soy Sauce', 'Tomato Sauce', 'Not Needed'],
        'Sweeteners': ['Sugar', 'Jaggery', 'Not Needed']
    }

    # Collect user input based on categories
    global selected_ingredients
    selected_ingredients = {}

    def on_ok_button_click(category, selected_ingredient):
        if selected_ingredient.lower() == 'not needed':
            print(f"\n{category} is not needed for the recipe.")
        else:
            print(f"\nSelected ingredient for {category} is: {selected_ingredient}")

            #(1) uncomment  beow line if user wants to select only 1 ingredient from each library
            #selected_ingredients[category] = selected_ingredient

            #(2)comment it if you uncomment above code
            selected_ingredients.setdefault(category, set()).add(selected_ingredient)



    for category, ingredients in categories.items():
        print(f"\n{category}: {', '.join(ingredients)}")
        dropdown = widgets.Dropdown(options=ingredients, description=f"Select {category.lower()} ingredients:")

        ok_button = widgets.Button(description="OK")
        ok_button.on_click(lambda b, category=category, dropdown=dropdown: on_ok_button_click(category, dropdown.value))

        #deplay horizontel buttons
        display(widgets.HBox([dropdown, ok_button]))

    # Use a button to signal when the user has finished selecting
    button = widgets.Button(description="Finished Selection")
    display(button)



    def on_button_click(b):
        global selected_ingredients  # Declare selected_ingredients as global
        print("\nFinal selected ingredients are:")
        print(selected_ingredients)

        if selected_ingredients:
            # Remove duplicates

            #(2) Comment it if you comment (2) above in on_ok_button_click function
            unique_selected_ingredients = []
            for ingredients_list in selected_ingredients.values():
              unique_selected_ingredients.extend(ingredients_list)

            #(1) uncomment if you uncomment above code in on_ok_button_click function
            #selected_ingredients = list(set(selected_ingredients.values()))

            #change (selected_ingredients) instead (unique_selected_ingredients) viseversa
            top_5_recipes, best_single_recipe = get_recipe_recommendations(unique_selected_ingredients, cosine_sim, df)

            if top_5_recipes:
              print('\nTop 5 Recommended Recipes:')
              for recipe in top_5_recipes:
                print('- ' + recipe)
            if best_single_recipe:
                print('\nSingle Best Recommended Recipe:')
                print('- ' + best_single_recipe[0])
            else:
                print('No single best recipe found.')

            if not top_5_recipes and not best_single_recipe:
                print('\nNo recipes found based on the selected ingredients.')
                print('Would you like to order food online?    Yes/No ?')

                #EDA Error not support this environment
                # order_choice = input('Enter "Yes" to order food online, or any key to exit: ')

                def open_online_food_ordering():
                    url = "https://www.ubereats.com/lk/city/colombo-western?psafe_param=1&utm_source=AdWords_NonBrand&utm_campaign=CM2052285-search-google-nonbrand_163_-99_LK-National_e_all_acq_cpc_en_DSA_Exact__dsa-943561206226_501944317344_108073870752__c&campaign_id=10969022555&adg_id=108073870752&fi_id=&match=&net=g&dev=c&dev_m=&ad_id=501944317344&cre=501944317344&kwid=dsa-943561206226&kw=&placement=&tar=&gclsrc=aw.ds&gclid=CjwKCAiAmsurBhBvEiwA6e-WPA2SOf_CS_aiLpJDBrRa9sniZxPnJEVa18OFksc6hA-H0OlU3McrzxoCiwgQAvD_BwE"

                    # Create a clickable link
                    link_html = f'<a href="{url}" target="_blank">Click here to order food online</a>'
                    display(HTML(link_html))

                def on_button_click(b):
                    if b.description.lower() == 'yes':
                        print('Redirecting to an online food ordering platform...')
                        open_online_food_ordering()
                    elif b.description.lower() == 'no':
                        print('Exiting. Have a good day!')

                # Create Yes and No buttons
                yes_button = widgets.Button(description="Yes")
                no_button = widgets.Button(description="No")

                # Attach the event handler to the buttons
                yes_button.on_click(on_button_click)
                no_button.on_click(on_button_click)

                # Display the buttons
                display(widgets.HBox([yes_button, no_button]))


    # Attach the event handler to the button widget
    button.on_click(on_button_click)

# Example usage: Recommend recipes based on user input
get_user_input_and_display_recommendations()


Available recipes:
['Rotti' 'Pasta' 'Pittu' 'Thosa' 'Idli' 'Vadai' 'Ulundhu Vadai' 'Uppuma'
 'Hopper (Aappa)' 'Noodles' 'Bun' 'Pizza' 'Patties' 'Pastry' 'Cake'
 'Pancake' 'Briyani' 'Fried Rice' 'Milk Rice (Kiri Badh)' 'Pongal'
 'Doughnut' 'Kanji' 'Sandwich' 'String Hopper' 'Rice' 'Pulses']

Proteins: Chicken, Egg, Not Needed



Vegetables: Carrot, Onion, Potato, Chilli, Coconut, Not Needed



Main: Wheat Flour, Pasta, Rice Flour, Rice, Bengal Gram, Ulundhu, Rava, Spaghetti, Bread, Chickpeas, Not Needed



Lentils: Green Gram, Kidney Beans, Not Needed



Dairy: Milk, Butter, Cheese, Not Needed


...


Sauces: Soy Sauce, Tomato Sauce, Not Needed



Sweeteners: Sugar, Jaggery, Not Needed


Button(description='Finished Selection', style=ButtonStyle())


Selected ingredient for Proteins is: Chicken

Selected ingredient for Vegetables is: Carrot

Main is not needed for the recipe.

Final selected ingredients are:
{'Proteins': {'Chicken'}, 'Vegetables': {'Carrot'}}

selected ingredients are : ['chicken', 'carrot']

main ingredients are : ['Wheat Flour', 'Pasta', 'Rice Flour', 'Rice', 'Bengal Gram', 'Ulundhu', 'Rava', 'Spaghetti', 'Bread', 'Chickpeas']

The main ingredient to make these recipes is not in the selected ingredients.
No single best recipe found.

No recipes found based on the selected ingredients.
Would you like to order food online?    Yes/No ?


Redirecting to an online food ordering platform...


In [ ]:
# select_button = widgets.Button(description="Yes")
# condition = True

# def on_ok_button_click(b):
#     if condition:
#         print('predssed')
#     else:
#         print('not pressed')

# ok_button.on_click(on_ok_button_click)
# display(ok_button)

Button(description='OK', style=ButtonStyle())